![image](https://hydro-jules.org/sites/default/files/Hydro-JULES_Logo_Positive.png)
# Hydro-JULES - Cf-python Examples

This notebook provides a very basic walk through of how to use a Datalabs python notebook, and how to open and read netCDF files in python using the Cf-Python and Cf-Plot packages. It shows how the spatial and time coordinates are commonly used within the netCDF files we have created for Hydro-JULES (which are relatively simple).

For an introduction of the tutorial please see the video https://youtu.be/dGif03kApJE 

Cf-Python documentation is avaialble at https://ncas-cms.github.io/cf-python/; Cf-Plot documentation is available at https://ajheaps.github.io/cf-plot/


**Contents** (note: to run cells later in the notebook, the first few cells must be run in order to import packages and initialise variables):
- [Import packages required for this work](#import_packages)
- [Quick netCDF file access to show how easy it is](#quick_file_access)
- [Different data printing options](#closer_look)
- [Reading the axis coordinates (X,Y,T)](#explore_coordinates)
- [Quick plotting of data to show how easy it is](#quick_plot_data)
- [Manipulating the data: Subspacing and Collapsing](#manipulating_data)
- [Further exercises](#further_exercises)
- [Cf-Python full tutorial](#explore_more)


<a id="import_packages"></a>
### Import packages required for this work

In [ ]:
# Importing cf-python and cfplot
import cf, cfplot as cfp

# Additional packages for advance plotting with cfplot
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

# Importing matplotlib to show that data read through cf-python can be plotted using matplotlib
import matplotlib.pyplot as plt

# The following line allows matplotlib plots to be shown in the notebook
%matplotlib inline

<a id="quick_file_access"></a>
### Quick netCDF file access to show how easy it is

In [ ]:
# Initialise a file name variable along with its path
path = '/data/example-data/'
file = 'chess-met_tas_gb_1km_daily_20151201-20151231.nc'

# Read the netCDF file using cf python
my_dataset = cf.read(path+file)

# See the netCDF file
my_dataset
# The following output is a list version of the file not a cf field. A cf field is in triangular brackets <> where as a list is in square brackets []

In [ ]:
# Converting list to a cf field by selecting the first item
my_dataset = my_dataset[0]
my_dataset
# The following output is a cf field rather than a list

In [ ]:
# Another method, to read in the cf file directly, is to add [0] when reading in the file
my_dataset = cf.read(path+file)[0]
my_dataset

# The first method should be used when you are not sure how many fields a netCDF file has, it is a good way to explore any dataset
# The second method should only be used when you absolutely sure that the netCDF file only has one field,
# As if the nerCDF file has more than one field using the second method will choose the first read fields and the fields are read randomly using cf.read
# cf.select option can also be used to read one particular variable or cf field from a list of cf fields
# For further detail please see https://ncas-cms.github.io/cf-python/tutorial.html#sorting-and-selecting-from-field-lists

<a id="closer_look"></a>
### Different data printing options

In [ ]:
#1 Shows you all the cf fields in brief
my_dataset


In [ ]:
#2 Shows you more details about the metadata of the cf fields
print(my_dataset)


In [ ]:
#3 Shows you maximum detail about the metadata of the cf fields
my_dataset.dump()


In [ ]:
#4 Prints out the data associated with the cf field (works only for one cf field)
print(my_dataset.data)


In [ ]:
#5 Prints out the data of the cf field as a numpy array (works only for one cf field)
print(my_dataset.array)


In [ ]:
# Method #5 can be used to convert a cf field data to a numpy array
my_numpy = my_dataset.array

# my_numpy is a numpy array having the data in the cf field and following is the data shape
print(my_numpy.shape)


<a id="explore_coordinates"></a>
### Reading the axis coordinates (X,Y,T)

In [ ]:
# Quickly see the file coordinates again
my_dataset


In [ ]:
# Reading the Y coordinate
my_dataset.coord('projection_y_coordinate')



In [ ]:
# Writing out the Y coordinate as an array
y = my_dataset.coord('projection_y_coordinate').array
print(y)

In [ ]:
# Reading the X coordinate
my_dataset.coord('projection_x_coordinate')


In [ ]:
# Writing out the X coordinate as an array
x = my_dataset.coord('projection_x_coordinate').array
print(x)


In [ ]:
# Reading the time coordinate
my_dataset.coord('time')


In [ ]:
print(my_dataset.coord('time').array)


In [ ]:
print(my_dataset.coord('time').dtarray)


<a id="quick_plot_data"></a>
### Quick plotting of data to show how easy it is

In [ ]:
#Cf-plot is the plotting software for cfpython
cfp.reset() # Just to reset any old settings for cfplot

#Plotting the first time step, same as the previous tutorial
cfp.mapset(resolution='50m') #This line is for improved resolution than default of 110m
cfp.con(my_dataset[0,:,:], lines=False, colorbar=True) # 1 Decmber 2015 air temperature plot
# cfp.con refers to contour plots, for more types of plots please see http://ajheaps.github.io/cf-plot/gallery.html
# my_dataset[0,:,:] uses subspacing by index, for more detail please see https://ncas-cms.github.io/cf-python/tutorial.html?#subspacing-by-index

<a id="manipulating_data"></a>
### Manipulating the data: Subspacing and Collapsing

Subspacing over time for 10 December 2015

In [ ]:
#Selecting time slice, for example selecting 10 December 2015
my_10dec = my_dataset.subspace(T = cf.year(2015) & cf.month(12) & cf.day(10)) 
my_10dec
# We do not need to subset for year 2015 and month 12 as the data only contains this month and year, just "my_dataset.subspace(T = cf.day(10))" will also work
# For other ways of time subspacing please see https://ncas-cms.github.io/cf-python/tutorial.html?#subspaces-in-time

In [ ]:
# Printing the time coordinate for the subset data
print(my_10dec.coord('time').dtarray)


In [ ]:
# Removing the single length axis
my_10dec = my_10dec.squeeze()
my_10dec

In [ ]:
# Plotting the air temperature for 10 December 2015
cfp.con(my_10dec, lines=False, colorbar=True, colorbar_orientation='vertical') # 10 December 2015 air temperature plot

Subspacing over area for southern England

In [ ]:
# Selecting an area within the whole domain, example southern England
my_area = my_10dec.subspace(projection_x_coordinate=cf.wi(350000,600000), projection_y_coordinate=cf.wi(50000,300000))
my_area
# For further details on collapsing over multiple dimensions please see https://ncas-cms.github.io/cf-python/tutorial.html?#multiple-dimensions

In [ ]:
# Plotting figure to compare full domain vs. selected domain
cfp.gopen(rows=1, columns=2) # Open a plot file to make multiple subplots. Please see "file" option in "cfp.gopen" to learn how to save plot as an image file
cfp.levs(min=270, max=285, step=1) # Defines colorbar levels
cfp.gpos(1) # First position of the subplot numbers defined in the cfp.gopen() command
cfp.con(my_10dec, lines=0, colorbar=0) # 10 December 2015 air temperature plot for the whole of UK
cfp.gpos(2)  # Second position of the subplot numbers defined in the cfp.gopen() command
cfp.con(my_area, lines=0, colorbar=1, colorbar_position=[0.2,0.2,0.6,0.02]) # 10 December 2015 air temperature plot for the southern England
cfp.gclose() # Plot needs to be closed to be rendered
# Colorbar levels are made common for both subplots and thus we only draw the colorbar once rather than drawing it with both subplots
# For more details on subplots please see http://ajheaps.github.io/cf-plot/multiple_plots.html

Collapsing over time for 25-31 December 2015

In [ ]:
# Subselecting time for 25-31 December 2015 
my_time = my_dataset.subspace(time = cf.day(cf.wi(25,31)))
my_time

In [ ]:
# Printing out the time coordinate
my_time.coord('time').dtarray


In [ ]:
# Averaging time over 25-31 December 2015 
my_time = my_time.collapse('time: mean')
my_time
# Collapse function can use many statistical routines like mean, standard deviation, minimum, maximum etc.
# For further details please see https://ncas-cms.github.io/cf-python/tutorial.html?#statistical-collapses

In [ ]:
# Plotting figure to compare different time slices
cfp.gopen(rows=2, columns=2)
cfp.levs(min=270, max=285, step=1)
cfp.cscale('viridis')
cfp.gpos(1)
cfp.con(my_10dec, lines=False, colorbar=0, title='10 Dec')
cfp.gpos(2)
cfp.con(my_time, lines=False, colorbar=1, title='25-31 Dec', colorbar_orientation='vertical', colorbar_position=[0.92,0.55,0.015,0.4])
cfp.levs(min=-3, max=3, step=0.5)
cfp.cscale('BlRe')
cfp.gpos(3)
cfp.con(my_10dec-my_time, lines=False, title='10 Dec minus 25-31 Dec', colorbar_orientation='vertical', colorbar_position=[0.5,0.07,0.015,0.4])
cfp.gclose()
# For further details on the colorscales and colorbars please see http://ajheaps.github.io/cf-plot/colour_scales.html


Collapsing over area for whole of UK and southern England

In [ ]:
# Averaging over the whole of UK
my_uk = my_dataset.collapse('area: mean')


In [ ]:
# Subselecting time for southern England again
my_se = my_dataset.subspace(projection_x_coordinate=cf.wi(350000,600000), projection_y_coordinate=cf.wi(50000,300000))
# Averaging over the whole southern England
my_se = my_se.collapse('area: mean')
my_se


In [ ]:
# We can also use data read through cf-python as numpy array and then plot using matplotlib
# Identify the time axis 
time = my_uk.coord('time').day.array

# Plotting line plots to compare the air temperature between whole of UK and southern England
fig = plt.figure(figsize=(18,7))
ax=plt.subplot(1,1,1)
plt.plot(time, my_uk.squeeze().array, color='black', label='UK')
plt.plot(time, my_se.squeeze().array, color='red', label='SE')
plt.ylabel('Air Temperature (Kelvin)')
plt.xticks(time)
plt.xlabel('Days in 2015 December')
plt.legend()
plt.show()
#For more information about using matplotlib please see https://matplotlib.org/


<a id="#further_exercises"></a>
### Further Exercises

If you are finished with the rest of the Notebook above, please try the following excercises in the cells given below the exercise.

1. Please plot the mean temperature for the whole of December 2015 as a spatial plot over UK in degree celsius. 
   
   HINT: 0Â°C = 273.15K

2. What is the mean temperature value for 17 December 2015 for the whole of UK?
   
   HINT: You have to average the temperature for all grid points for UK. 

3. Compare the temperature difference between London and Edinburgh 

   HINT: Use Easting (X) = 529500 and Northing (Y) = 179500 for London | Easting (X) = 332500 and Northing (Y) = 673500 for Edinburgh

4. What all is wrong with the code below? Debug the code in the cell below and make sure it provides the correct answer. Please consider the comment in the cell as the truth. 

   HINT: Please pay attention to the error messages displayed. Further, the code running without warnings or errors does not mean the answer is correct, please check if the calculated output and overall plot is correct. 
   

In [ ]:
#Calculating the temperature difference between 30 December and 1 December 2015 and plotting one figure with all values and one only potting the differences less than 4Kelvin
my_01dec = my_dataset.subspace(time=cf.day(1))
my_30dec = my_dataset.subspace(time=cf.day(31)).squeeze()
my_diff = my_30dec - my_01dec
new_diff = my_diff.where(cf.gt(4), cf.masked)

cfp.reset()
cfp.gopen(rows=1, columns=2)
cfp.cscale('scale1)
cfp.levs(min=-5, max=5, step=1)
cfp.gpos(1)
cfp.con(new_diff, lines=0, colorbar=False, title='All values')
cfp.gpos(3)
cfp.con(my_diff, lines=0, title='Values less than 4deg difference', colorbar=False, colorbar_title='Air Temperature Difference', colorbar_orientation='horizontal', colorbar_position=[0.12,0.22,0.8,0.02])
cfp.gclose()


5. What all is wrong with the code below? Debug the code in the cell below and make sure it provides the correct answer. Please consider the comment in the cell as the truth. 

   HINT: Please pay attention to the error messages displayed. Further, the code running without warnings or errors does not mean the answer is correct, please check if the calculated output and overall plot is correct. 

In [ ]:
#Calculating and plotting mean, minimum, maximum, standard deviation of the air temperature over whole of December 2015 for each grid point
my_mean = my_dataset.collapse('time: mean')
my_max  = my_dataset.collapse('time: minimum').squeeze()
my_min  = my_dataset.collapse('time: maximum').squeeze()
my_sd   = my_time.collapse('time: sd').squeeze()

cfp.reset()
cfp.gopen(rows=1, columns=2)
cfp.gpos(1)
cfp.cscale('plasma')
cfp.con(my_mean, lines=0, colorbar=True, title='(b) Mean', colorbar_orientation='vertical', colorbar_title='')
cfp.gpos(1)
cfp.cscale('viridis')
cfp.con(my_std, lines=0, colorbar=False, title='(a) Standard Deviation', colorbar_title='')
cfp.levs(min=265, max=289, step=0.5)
cfp.gpos(3)
cfp.cscale('parula')
cfp.con(my_min, lines=0, colorbar=True, title='(c) Minium', colorbar_orientation='vertical', colorbar_title='')
cfp.gpos(4)
cfp.cscale('parala')
cfp.con(my_max, lines=0, colorbar=True, title='(d) Minimum', colorbar_orientation='vertical')
cfp.gclose()



<a id="explore_more"></a>
### Cf-Python full tutorial

Cf-python tutorial documentation is avaialble at https://ncas-cms.github.io/cf-python/tutorial.html

Full training and short courses are avaialble at https://github.com/NCAS-CMS/cf-training
